In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/CS280_final_project/hands_clapping
%ls

/content/drive/MyDrive/CS280_final_project/hands_clapping
5.5_transformer_nature_loss.npy           hands_clapping_2.MOV
5.6_transformer_clap_aud_out.mp3          hands_clapping_3_test.MOV
5.7_transformer_clap_aud_out.mp3          hands_clapping.MOV
5.7_transformer_clap_aud_out_valid_2.mp3  larger_vid_arr_2.mp4
5.7_transformer_clap_aud_out_valid.mp3    larger_vid_arr_3.mp4
5.7_transformer_clap_model_WORKING        larger_vid_arr.mp4
5.7_transformer_clap_model_WORKING_2      larger_vid_arr_with_sound_3.mp4
5.8_transformer_clap_aud_out_3_test.mp3   matt_clap.MOV
clapping_loss_v1.npy                      __temp__.mp4
clapping_v1_aud_out.npy                   test_audio_2.mp3
clapping_v1_aud_test.mp3                  test_clap_4.6.mp4
clappingv1.pt                             test_model_out_5.2_1.mp3
DEMO2_larger_vid_arr_with_sound_2.mp4     test.mp4
DEMOV1_larger_vid_arr_with_sound.mp4      transformer_8hrnature_5.5.pt


In [ ]:
!sudo pip install scikit-video

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import cv2
import soundfile as sf
import numpy as np
import skvideo.io  
import matplotlib.pyplot as plt

In [ ]:
video_string = 'matt_clap.MOV'

In [ ]:
#NOTE: use vreader and then create a downsampled np array
resized_vid_arr = []
videodata = skvideo.io.vreader(video_string)  

count = 0 
for v in videodata:
  count += 1
  if count % 3 == 0:
    roi = cv2.resize(v, (24, 24))
    roi = roi.astype("float") / 255.0
    resized_vid_arr.append(roi) 
resized_vid_arr = np.array(resized_vid_arr)

In [ ]:
from moviepy.video.io.VideoFileClip import AudioFileClip
audioclip = AudioFileClip(video_string, fps=8000)
# audioclip = videoclip.audio
audio_array = audioclip.to_soundarray()

  return stacker(self.iter_chunks(fps=fps, quantize=quantize,



In [ ]:
print(audio_array[0])

[0.00515747 0.00408936]


In [ ]:
!pip install pydub
import pydub 
fps = 10

def write(f, sr, x, normalized=True):
    """numpy array to MP3"""
    channels = 2 if (x.ndim == 2 and x.shape[1] == 2) else 1
    if normalized:  # normalized array - each item should be a float in [-1, 1)
        y = np.int16(x * 2 ** 15)
    else:
        y = np.int16(x)
    song = pydub.AudioSegment(y.tobytes(), frame_rate=sr, sample_width=2, channels=channels)
    song.export(f, format="mp3", bitrate="320k")

# sr = (audio_array.shape[0]// resized_vid_arr.shape[0])*fps
# write('test_audio_2.mp3', sr, audio_array)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
print(resized_vid_arr.shape)
print(audio_array.shape)
print(audio_array.shape[0]//resized_vid_arr.shape[0])
print(audio_array.shape[0]/resized_vid_arr.shape[0])

(247, 24, 24, 3)
(197600, 2)
800
800.0


In [ ]:
print(0.9*audio_array.shape[0], int(0.9*audio_array.shape[0]))
print(0.9*resized_vid_arr.shape[0], int(0.9*resized_vid_arr.shape[0]))

177840.0 177840
222.3 222


In [ ]:
valid_resized_vid_arr = resized_vid_arr[int(0.9*resized_vid_arr.shape[0]):]
valid_aud_arr = audio_array[int(0.9*audio_array.shape[0]):]

resized_vid_arr = resized_vid_arr[:int(0.9*resized_vid_arr.shape[0])]
audio_array = audio_array[:int(0.9*audio_array.shape[0])]

In [ ]:
print(resized_vid_arr.shape, valid_resized_vid_arr.shape)

(222, 24, 24, 3) (25, 24, 24, 3)


In [ ]:
print(audio_array.shape, valid_aud_arr.shape)

(177840, 2) (19760, 2)


In [ ]:
resized_vid_arr = np.concatenate((resized_vid_arr, resized_vid_arr, resized_vid_arr, resized_vid_arr, resized_vid_arr, resized_vid_arr), axis = 0)
audio_array = np.concatenate((audio_array, audio_array, audio_array, audio_array, audio_array, audio_array), axis = 0)

valid_resized_vid_arr = np.concatenate((valid_resized_vid_arr, valid_resized_vid_arr, valid_resized_vid_arr, valid_resized_vid_arr), axis = 0)
valid_aud_arr = np.concatenate((valid_aud_arr, valid_aud_arr, valid_aud_arr, valid_aud_arr), axis = 0)

*training loop from cs189 hw*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import gc
from tqdm.notebook import tqdm, trange

In [ ]:
class VideoAudioDataset(Dataset):
    def __init__(self, video_frames, audio_frames, num_frames, aud_per_frame):
        self.video_frames = torch.tensor(video_frames, dtype=torch.float32).permute(0,3,2,1) # Permute to (N, C, H, W)
        self.aud_per_frame = aud_per_frame
        aud_clip_amount = audio_frames.shape[0] % self.aud_per_frame
        print(self.aud_per_frame, aud_clip_amount)
        if aud_clip_amount > 0:
          self.audio_frames = torch.tensor(audio_frames[:-aud_clip_amount], dtype=torch.float32).reshape(-1, self.aud_per_frame,audio_frames.shape[1]).permute(0,2,1) # Permute to (N, C, A)
        else:
          self.audio_frames = torch.tensor(audio_frames, dtype=torch.float32).reshape(-1, self.aud_per_frame,audio_frames.shape[1]).permute(0,2,1) # Permute to (N, C, A)
        print(self.audio_frames.shape)
        self.num_frames = num_frames

        num_vid_frames =  self.audio_frames.shape[0]
        # print(self.audio_frames.shape[0])
        # print(aud_per_frame)
        # print(num_vid_frames)
        self.video_frames = self.video_frames[:num_vid_frames]

    def __len__(self):
        return len(self.video_frames)

    def __getitem__(self, idx):
        idx += 1
        if idx < self.num_frames:
          num_zeros_needed = self.num_frames - idx
          vid_zeros = torch.zeros(num_zeros_needed, *self.video_frames[0].shape)
          aud_zeros = torch.zeros(num_zeros_needed, *self.audio_frames[0].shape)
          vid = torch.vstack((vid_zeros, self.video_frames[0:idx])).transpose(0,1)
          aud = torch.vstack((aud_zeros, self.audio_frames[0:idx])).transpose(0,1).reshape(2,-1)
          return (vid, aud)
        #vid shape example torch.Size([32, 3, 10, 36, 64])
        # aud shape example torch.Size([32, 2, 14710])
        vid = self.video_frames[idx-self.num_frames:idx].transpose(0,1)
        aud = self.audio_frames[idx-self.num_frames: (idx)].transpose(0,1).reshape(2,-1)
        # print('idx = ', idx, ' aud.size = ', aud.shape)
        return (vid, aud)

In [ ]:
int(0.9*resized_vid_arr.shape[0])

1198

In [ ]:
audio_array.shape, resized_vid_arr.shape

((1067040, 2), (1332, 24, 24, 3))

In [ ]:
valid_aud_arr.shape, valid_resized_vid_arr.shape

((79040, 2), (100, 24, 24, 3))

In [ ]:
train_data = VideoAudioDataset(resized_vid_arr, audio_array, num_frames = 10, aud_per_frame = 800)
valid_data = VideoAudioDataset(valid_resized_vid_arr, valid_aud_arr, num_frames = 10, aud_per_frame = 800)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=True)

800 640
torch.Size([1333, 2, 800])
800 640
torch.Size([98, 2, 800])


In [ ]:
for v in valid_data:
  print(v)
  break

In [ ]:
for v in valid_loader:
  print(v)
  break

https://github.com/antecessor/Wavenet << source!

In [ ]:
class AudConvEmbedding(nn.Module):
  def __init__(self, in_channels, output_dim):
    super(AudConvEmbedding, self).__init__()
    self.conv_layers = nn.Sequential(
        nn.Conv1d(in_channels = 2, out_channels = 2, kernel_size = 5, stride = 2),
        nn.LayerNorm(3998),
        nn.Conv1d(in_channels = 2, out_channels = 2, kernel_size = 5, stride = 2),
        nn.LayerNorm(1997),
        nn.Conv1d(in_channels = 2, out_channels = 2, kernel_size = 5, stride = 2),
        nn.LayerNorm(997),
        # nn.Conv1d(in_channels = 2, out_channels = 2, kernel_size = 5, stride = 2),
        # nn.LayerNorm(992),
        # nn.Conv1d(in_channels = 2, out_channels = 2, kernel_size = 5, stride = 2),
        # nn.LayerNorm(494),
        nn.Linear(997, output_dim)
    )
  
  def forward(self, x):
    return self.conv_layers(x)

In [ ]:
class MultiheadAttention(nn.Module):

    def __init__(self, input_dim, embed_dim, num_heads, dropout):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dimension must be 0 modulo number of heads."

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        self.dropout = nn.Dropout(dropout)

        # Stack all weight matrices 1...h together for efficiency
        # Note that in many implementations you see "bias=False" which is optional
        self.qkv_proj = nn.Linear(input_dim, 3*embed_dim)
        self.o_proj = nn.Linear(embed_dim, input_dim)

        self._reset_parameters()

    def _reset_parameters(self):
        # Original Transformer initialization, see PyTorch documentation
        nn.init.xavier_uniform_(self.qkv_proj.weight)
        self.qkv_proj.bias.data.fill_(0)
        nn.init.xavier_uniform_(self.o_proj.weight)
        self.o_proj.bias.data.fill_(0)

    def scaled_dot_product(self, q, k, v, mask=None):
        d_k = q.size()[-1]
        attn_logits = torch.matmul(q, k.transpose(-2, -1))
        attn_logits = attn_logits / np.sqrt(d_k)
        if mask is not None:
            attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
        attention = F.softmax(attn_logits, dim=-1)
        attention = self.dropout(attention)
        values = torch.matmul(attention, v)
        return values, attention

    def forward(self, x, mask=None, return_attention=False):
        batch_size, seq_length, _ = x.size()
        qkv = self.qkv_proj(x)

        # Separate Q, K, V from linear output
        qkv = qkv.reshape(batch_size, seq_length, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3) # [Batch, Head, SeqLen, Dims]
        q, k, v = qkv.chunk(3, dim=-1)
        #print(q.shape, k.shape, v.shape)
        # Determine value outputs
        values, attention = self.scaled_dot_product(q, k, v, mask=mask)
        values = values.permute(0, 2, 1, 3) # [Batch, SeqLen, Head, Dims]
        #print(values.shape)
        values = values.reshape(batch_size, seq_length, self.embed_dim)
        o = self.o_proj(values)

        if return_attention:
            return o, attention
        else:
            return o

In [ ]:
class AddPosEncoding(nn.Module):
  def __init__(self, d_model = 256, input_dropout = 0.1, timing_dropout = 0.1, max_len = 512):
    super(AddPosEncoding, self).__init__()
    self.d_model = d_model
    self.input_dropout = input_dropout
    self.timing_dropout = timing_dropout
    self.max_len = max_len

    self.timing_table = nn.Parameter(torch.FloatTensor(max_len, d_model))
    nn.init.normal_(self.timing_table)
    self.input_dropout = nn.Dropout(input_dropout)
    self.timing_dropout = nn.Dropout(self.timing_dropout)

  
  def forward(self,x):
    x = self.input_dropout(x)
    timing = self.timing_table[None, :x.shape[1], :]
    timing = self.timing_dropout(timing)
    return x + timing 



In [ ]:
class AudTransformer(nn.Module):
  def __init__(self, in_channels, out_channels, hidden_dims = 256, seq_len = 100, dim_ff = 1024, n_layers = 6, n_head = 8, d_qkv = 64, dropout = 0.1):
        super(AudTransformer, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.hidden_dims = hidden_dims
        self.dim_ff = dim_ff
        self.n_layers = n_layers
        self.n_head = n_head
        self.d_qkv = d_qkv
        self.dropout = dropout

        self.mha_list = nn.ModuleList()
        self.mha_norms = nn.ModuleList()
        self.pff_list = nn.ModuleList()
        self.pff_norms = nn.ModuleList()

        self.output_norm = nn.LayerNorm(hidden_dims)
        self.linear = nn.Linear(self.hidden_dims, 2)
        self.tanh = nn.Tanh()

        self.add_timing = AddPosEncoding(hidden_dims, max_len = seq_len)

        self.embedding = AudConvEmbedding(2, self.hidden_dims)
        self.embedding2 = nn.Linear(2, self.hidden_dims)


        for _ in range(n_layers):
          self.mha_list.append(MultiheadAttention(self.hidden_dims, self.d_qkv, self.n_head, self.dropout))
          self.mha_norms.append(nn.LayerNorm(hidden_dims))
          self.pff_list.append(nn.Sequential(
                              nn.Linear(hidden_dims, dim_ff),
                              nn.ReLU(),
                              nn.Linear(dim_ff, hidden_dims),
                              nn.Dropout(self.dropout)
                              ))
          self.pff_norms.append(nn.LayerNorm(hidden_dims))

  def forward(self, x):
    x = self.embedding(x).transpose(2,1)
    x = self.embedding2(x)
    x = x + self.add_timing(x)
    for i in range(self.n_layers):
      x_norm = self.mha_norms[i](x)
      #print('x_norm.shape', x_norm.shape)
      att_out = self.mha_list[i](x)
      #print(att_out.shape)
      x = x_norm + att_out
      x_norm = self.pff_norms[i](x)
      ff_out = self.pff_list[i](x_norm)
      x = x_norm + ff_out

    x = self.output_norm(x)
    x = self.linear(x)
    x = self.tanh(x)
    return x


In [ ]:
class VidToAudFusion(nn.Module):
    def __init__(self, audio_dim, video_dims, audio_channels, vid_channels, dropout_prob=0.1, num_frames=10):
        super(VidToAudFusion, self).__init__()
        self.dim_proj = nn.Linear(576, audio_dim)
        print(audio_dim)
        self.audio_dim = audio_dim
        self.video_dims = video_dims
        self.vid_channels = vid_channels


        if audio_channels == vid_channels:
            self.channel_projection = nn.Identity()
        else:
            self.channel_projection = nn.Sequential(
                nn.Conv2d(vid_channels, audio_channels, kernel_size=1, stride=1, bias=True),
            )
        self.channel_projection2 = nn.Sequential(
                nn.Conv1d(num_frames, 2, kernel_size=1, stride=1, bias=True),
            )
            
    def forward(self, x):
      #x = x.reshape(x.shape[0], x.shape[1], x.shape[2], -1)
      x = torch.flatten(x, start_dim=3)
      # print(x.shape)
      audio_proj = self.dim_proj(x)
      #print(audio_proj.shape)
      audio_proj = self.channel_projection(audio_proj).squeeze()
      audio_proj = self.channel_projection2(audio_proj)


      #print(audio_proj.shape)
      return audio_proj

In [ ]:
class ConvResBlock3D(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_prob=0.1, stride=1, kernel_size=3, padding=1):
        super(ConvResBlock3D, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=True)
        self.norm1 = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=kernel_size, stride=1, padding=padding, bias=True)
        self.norm2 = nn.BatchNorm3d(out_channels)
        self.stride = stride
        
        if in_channels == out_channels:
            self.skip_connection = nn.Identity()
        else:
            self.skip_connection = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm3d(out_channels)
            )
            
    def forward(self, x):
        residual = self.skip_connection(x)
        
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.conv2(out)
        out = self.norm2(out)
        out = self.dropout(out)
        
        out = self.relu(out)
        out = out + residual

        return out

class ConvResBlock2D(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_prob=0.1, stride=1, kernel_size=3, padding=1):
        super(ConvResBlock2D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=True)
        self.norm1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=1, padding=padding, bias=True)
        self.norm2 = nn.BatchNorm2d(out_channels)
        self.stride = stride
        
        if in_channels == out_channels:
            self.skip_connection = nn.Identity()
        else:
            self.skip_connection = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(out_channels)
            )
            
    def forward(self, x):
        residual = self.skip_connection(x)
        
        out = self.conv1(x)
        out = self.norm1(out)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.conv2(out)
        out = self.norm2(out)
        out = self.dropout(out)
        
        out = self.relu(out)
        out = out + residual

        return out

In [ ]:
### combines video into audio input vector and then passes through wavenet

class AttAudVideoNet(nn.Module):
    def __init__(self,audio_input_shape, video_input_shape,in_channels=2,out_channels=2): ## NEED TO DEBUG THESE hyperperams 4.29
        super().__init__()
        self.transformer=AudTransformer(in_channels, out_channels, hidden_dims = 64, seq_len = 8000, dim_ff = 1024, n_layers = 6, n_head = 8, d_qkv = 64, dropout = 0.1) 
        self.tanh = nn.Tanh()
        self.vid_convs = nn.ModuleList([
            ConvResBlock3D(3, 16),
            ConvResBlock3D(16, 32),
            ConvResBlock3D(32, 64)
        ])
        self.audio_input_shape = audio_input_shape
        self.video_input_shape = video_input_shape
        vid_dims_list = self.get_video_dims(video_input_shape)
        self.vid_to_aud = VidToAudFusion(self.audio_input_shape[-1], vid_dims_list[2], 1, 64, num_frames=video_input_shape[2])
        
        self.lin_layer = nn.Linear(128, 2)
        

    def get_video_dims(self, video_input_shape):
        shape_list = []
        test_data = torch.ones(video_input_shape)
        out = self.vid_convs[0](test_data)
        shape_list.append((out.shape[-2],out.shape[-1]))
        for layer in self.vid_convs[1:]:
            out = layer(out)
            shape_list.append((out.shape[-2],out.shape[-1]))
        return shape_list

    def forward(self,vid, aud):
        # print('aud_shape init = ', aud.shape)
        # print('vid shape init = ', vid.shape)
        for i in range(len(self.vid_convs)):
          vid = self.vid_convs[i](vid)
        vid_to_aud = self.vid_to_aud(vid)
        # print('vid to aud vs aud shape = ', vid_to_aud.shape, aud.shape)
        aud = aud + vid_to_aud
        
        aud=self.transformer(aud)
        # print('aud.shape', aud.shape)
        flattened_aud = aud.view(aud.shape[0], -1)
        aud = self.lin_layer(flattened_aud)
        return self.tanh(aud)



In [ ]:
for x in train_loader:
  print(x[0].shape)
  print(x[1].shape)
  break
  
count = 0
# for x in valid_loader:
#   count += 1
#   if count == 10:
#     print(x[0].shape)
#     print(x[1].shape)
#     break

In [ ]:
from tqdm.notebook import tqdm, trange

# wavenet = WaveNet(in_channels=2,out_channels=2,kernel_size=2,stack_size=23,layer_size=6).cuda().train()
audio_test_data = torch.ones((1,2,7999))
vid_test_data = torch.ones((1,3,10,24,24))

wavenet = AttAudVideoNet(audio_input_shape=audio_test_data.shape, video_input_shape=vid_test_data.shape,in_channels=2,out_channels=2).cuda().train()


optimizer=torch.optim.AdamW(wavenet.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# lossFunction = torch.nn.L1Loss()
lossFunction = nn.CrossEntropyLoss()

def calc_accuracy(Out,Y):
    max_vals, max_indices = torch.max(Out,1)
    train_acc = (max_indices == Y).sum().item()/max_indices.size()[0]
    return train_acc
  
epochs= 30
globalStep=500

valid_loss_list = []

for epoch in range(epochs):
    for step, (vid_frames,aud_frames) in tqdm(enumerate(train_loader),desc="Training"):
         #vid_frames = vid_frames.cuda()
         target = aud_frames[:,:,-1].cuda()
         aud_frames = aud_frames[:,:,:-1].cuda()
         vid_frames = vid_frames.cuda()
         #print(target.shape)
        #  print(aud_frames.shape)
        #  print(vid_frames.shape)
         output = wavenet(vid_frames, aud_frames).squeeze()
         #print(output[0].detach().cpu().numpy(), target[0].cpu().numpy())
         #print(output.shape)
         #print(output)
         #print(output.dtype, target.dtype)

         loss = lossFunction(output,target)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
         if step%globalStep==0:
            scheduler.step()
            # print(output.detach().numpy())
            # print(y_train.numpy())
            with torch.no_grad():
                accuracy=0
                val_loss=0
                for stepTest, (vid_frames,aud_frames) in tqdm(enumerate(valid_loader),desc="Validation"):
                    vid_frames = vid_frames.cuda()
                    target = aud_frames[:,:,-1].cuda()
                    aud_frames = aud_frames[:,:,:-1].cuda()
                    if True:
                      # print('aud and vid frames shapes')
                      # print(aud_frames.shape)
                      # print(vid_frames.shape)
                      output = wavenet(vid_frames, aud_frames).squeeze()

                      #accuracy+=calc_accuracy(output,target)*100
                      val_loss+= lossFunction(output,target).item()
                    if step == 0:
                      print(target[:3])
                      print(output[:3])
                    if stepTest>200:
                        break
            print(f"loss for step {step} : {val_loss/stepTest}")
            valid_loss_list.append(val_loss/stepTest)

         
    print(f"epch {epoch}")


7999


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

tensor([[-0.0378, -0.0391],
        [-0.0122, -0.0107],
        [-0.0042,  0.0000]], device='cuda:0')
tensor([[ 0.3713, -0.5612],
        [ 0.3534, -0.5491],
        [ 0.3516, -0.5657]], device='cuda:0')
tensor([[-0.0473, -0.0600],
        [ 0.0375,  0.0419],
        [-0.0006,  0.0060]], device='cuda:0')
tensor([[ 0.2584, -0.5162],
        [-0.0837, -0.2012],
        [ 0.2619, -0.5834]], device='cuda:0')
tensor([[ 0.0188,  0.0272],
        [-0.0053, -0.0012],
        [-0.0126, -0.0141]], device='cuda:0')
tensor([[ 0.2950, -0.5814],
        [ 0.2928, -0.5779],
        [ 0.3136, -0.5936]], device='cuda:0')
tensor([[ 0.0109,  0.0128],
        [-0.0002, -0.0041]], device='cuda:0')
tensor([[ 0.3827, -0.4606],
        [ 0.4010, -0.4987]], device='cuda:0')
loss for step 0 : 0.012657086364924908
epch 0


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

tensor([[ 0.0122,  0.0174],
        [-0.0017, -0.0037],
        [ 0.0535,  0.0577]], device='cuda:0')
tensor([[-0.9329,  0.9459],
        [-0.9453,  0.9453],
        [-0.9466,  0.9490]], device='cuda:0')
tensor([[ 0.0303,  0.0371],
        [-0.0441, -0.0181],
        [-0.0039, -0.0086]], device='cuda:0')
tensor([[-0.9548,  0.9342],
        [-0.9467,  0.9486],
        [-0.9462,  0.9491]], device='cuda:0')
tensor([[ 0.0306,  0.0331],
        [-0.0105, -0.0119],
        [-0.0053, -0.0012]], device='cuda:0')
tensor([[-0.9468,  0.9492],
        [-0.9471,  0.9492],
        [-0.9465,  0.9588]], device='cuda:0')
tensor([[0.0186, 0.0185],
        [0.0069, 0.0078]], device='cuda:0')
tensor([[-0.9466,  0.9489],
        [-0.9548,  0.9340]], device='cuda:0')
loss for step 0 : 0.02236494484047095
epch 28


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

tensor([[ 0.0199,  0.0528],
        [ 0.0011,  0.0008],
        [-0.0284, -0.0425]], device='cuda:0')
tensor([[-0.9418,  0.9526],
        [-0.9475,  0.9491],
        [-0.9472,  0.9490]], device='cuda:0')
tensor([[ 0.0036,  0.0069],
        [-0.0148, -0.0206],
        [ 0.0088,  0.0108]], device='cuda:0')
tensor([[-0.9475,  0.9487],
        [-0.9467,  0.9493],
        [-0.9474,  0.9487]], device='cuda:0')
tensor([[-0.0126, -0.0141],
        [-0.0029, -0.0071],
        [-0.0117, -0.0117]], device='cuda:0')
tensor([[-0.9476,  0.9493],
        [-0.9468,  0.9493],
        [-0.9468,  0.9494]], device='cuda:0')
tensor([[-0.0042,  0.0000],
        [ 0.0223,  0.0274]], device='cuda:0')
tensor([[-0.9556,  0.9421],
        [-0.9473,  0.9491]], device='cuda:0')
loss for step 0 : 0.020084032012770574
epch 29


In [ ]:
class VideoOnlyDataset(Dataset):
    def __init__(self, video_frames, num_frames):
        self.video_frames = torch.tensor(video_frames, dtype=torch.float32).permute(0,3,2,1) # Permute to (N, C, H, W)
        self.num_frames = num_frames

    def __len__(self):
        return len(self.video_frames)

    def __getitem__(self, idx):
        idx += 1
        if idx < self.num_frames:
          num_zeros_needed = self.num_frames - idx
          vid_zeros = torch.zeros(num_zeros_needed, *self.video_frames[0].shape)
          vid = torch.vstack((vid_zeros, self.video_frames[0:idx])).transpose(0,1)
          return vid
        #vid shape example torch.Size([32, 3, 10, 36, 64])
        # aud shape example torch.Size([32, 2, 14710])
        vid = self.video_frames[idx-self.num_frames:idx].transpose(0,1)
        # print('idx = ', idx, ' aud.size = ', aud.shape)
        return vid

In [ ]:
# ### generate audio for video from model
# num_frames = 10

# start_index = int(len(resized_vid_arr)*0.8) + num_frames
# audio_list = []
# vid_test_shape = (1,3,10,24,24)
# zero_frame = torch.zeros((vid_test_shape[3], vid_test_shape[4]))

# aud_per_vid_frame = audio_array.shape[0]//resized_vid_arr.shape[0]
# audio_start_index = int(len(audio_array)*0.8)
# print(audio_start_index)
# aud_input_arr = torch.tensor(audio_array[audio_start_index:audio_start_index+(num_frames * aud_per_vid_frame) - 1], dtype=torch.float32).cuda().T.unsqueeze(0)
# print(aud_input_arr.shape)

# #aud_input_arr = torch.zeros((1,2,(num_frames * aud_per_vid_frame) - 1)).cuda()
# print(aud_input_arr.shape)

# input_vid = resized_vid_arr[start_index:]
# gen_data = VideoOnlyDataset(input_vid, num_frames = num_frames)
# gen_loader = DataLoader(gen_data, batch_size=1, shuffle=False)


# max_count = 30

# #NOTE: GEN NEEDS TO BE FIXED TO ACCOUNT FOR MULTIPLE AUDIO RUNS PER FRAME
# wavenet.eval()
# for i,vid in enumerate(gen_loader):
#     vid = vid.cuda()
#     print(vid.shape)
#     if i >= max_count:
#       break
#     for j in trange(aud_per_vid_frame):
#         with torch.no_grad():
#             # print(aud_input_arr.shape)
#             # print(vid.shape)
#             audio_output = wavenet(vid.cuda(), aud_input_arr.cuda()).cpu()
#         audio_list.append(audio_output.squeeze().cpu().numpy())
#         aud_input_arr = aud_input_arr[:, :, 1:]
#         aud_input_arr = torch.cat((aud_input_arr, audio_output.cuda().unsqueeze(2)), 2)

# np.array(audio_list).shape


# audio_np = np.array(audio_list).transpose(1,0)
# print(np.array(audio_list).shape)
# audio_np = np.array(audio_list).transpose(1,0)
# fps = 10
# sr = (audio_array.shape[0]// resized_vid_arr.shape[0])*fps
# write('5.7_transformer_clap_aud_out.mp3', sr, audio_np)


In [ ]:
torch.save(wavenet.state_dict(), '5.8_transformer_clap_model_WORKING_3')

In [ ]:
audio_test_data = torch.ones((1,2,7999))
vid_test_data = torch.ones((1,3,10,24,24))
wavenet = AttAudVideoNet(audio_input_shape=audio_test_data.shape, video_input_shape=vid_test_data.shape,in_channels=2,out_channels=2).cuda()
wavenet.load_state_dict(torch.load('5.8_transformer_clap_model_WORKING_3'))
wavenet.eval()

Now need to reconstruct the video somehow!

In [ ]:
## get the data again
#NOTE: use vreader and then create a downsampled np array
resized_vid_arr = []
larger_vid_arr = []
videodata = skvideo.io.vreader(video_string)  

count = 0 
for v in videodata:
  count += 1
  if count % 3 == 0:
    roi = cv2.resize(v, (24, 24))
    roi = roi.astype("float") / 255.0
    resized_vid_arr.append(roi) 
    larger_vid_arr.append(v)
resized_vid_arr = np.array(resized_vid_arr)

from moviepy.video.io.VideoFileClip import AudioFileClip
audioclip = AudioFileClip(video_string, fps=8000)
# audioclip = videoclip.audio
audio_array = audioclip.to_soundarray()

  return stacker(self.iter_chunks(fps=fps, quantize=quantize,



In [ ]:
print(resized_vid_arr[0].shape)

(24, 24, 3)


In [ ]:
num_frames = 10

start_percent = 0.0

start_index = int(len(resized_vid_arr)*start_percent) + num_frames
audio_list = []
vid_test_shape = (1,3,10,24,24)
zero_frame = torch.zeros((vid_test_shape[3], vid_test_shape[4]))

aud_per_vid_frame = audio_array.shape[0]//resized_vid_arr.shape[0]
audio_start_index = int(len(audio_array)*start_percent)
print(audio_start_index)
aud_input_arr = torch.tensor(audio_array[audio_start_index:audio_start_index+(num_frames * aud_per_vid_frame) - 1], dtype=torch.float32).cuda().T.unsqueeze(0)
print(aud_input_arr.shape)

#aud_input_arr = torch.zeros((1,2,(num_frames * aud_per_vid_frame) - 1)).cuda()
print(aud_input_arr.shape)

input_vid = resized_vid_arr[start_index:]
larger_vid_arr = larger_vid_arr[start_index:]
gen_data = VideoOnlyDataset(input_vid, num_frames = num_frames)
gen_loader = DataLoader(gen_data, batch_size=1, shuffle=False)

video_list = []

max_count = 30

#NOTE: GEN NEEDS TO BE FIXED TO ACCOUNT FOR MULTIPLE AUDIO RUNS PER FRAME
wavenet.eval()
for i,vid in enumerate(gen_loader):
    vid = vid.cuda()
    print(vid.shape)
    if i >= max_count:
      break
    video_list.append(larger_vid_arr[i])
    for j in trange(aud_per_vid_frame):
        with torch.no_grad():
            # print(aud_input_arr.get_device())
            # print(vid.get_device())
            audio_output = wavenet(vid.cuda(), aud_input_arr.cuda()).cpu()
        audio_list.append(audio_output.squeeze().cpu().numpy())
        aud_input_arr = aud_input_arr[:, :, 1:]
        aud_input_arr = torch.cat((aud_input_arr, audio_output.cuda().unsqueeze(2)), 2)

np.array(audio_list).shape


video_np = np.array(video_list)
audio_np = np.array(audio_list).transpose(1,0)
print(np.array(audio_list).shape)
audio_np = np.array(audio_list).transpose(1,0)
fps = 10
sr = (audio_array.shape[0]// resized_vid_arr.shape[0])*fps
write('5.8_transformer_clap_aud_out_4_test.mp3', sr, audio_np)

0
torch.Size([1, 2, 7999])
torch.Size([1, 2, 7999])
torch.Size([1, 3, 10, 24, 24])


  0%|          | 0/800 [00:00<?, ?it/s]

torch.Size([1, 3, 10, 24, 24])


  0%|          | 0/800 [00:00<?, ?it/s]

torch.Size([1, 3, 10, 24, 24])


...

torch.Size([1, 3, 10, 24, 24])


  0%|          | 0/800 [00:00<?, ?it/s]

torch.Size([1, 3, 10, 24, 24])


  0%|          | 0/800 [00:00<?, ?it/s]

torch.Size([1, 3, 10, 24, 24])
(24000, 2)


In [ ]:
print(video_np.shape, audio_np.shape)

(30, 1080, 1920, 3) (2, 24000)


In [ ]:
# ## get the last 80% of the frames from the video
# larger_vid_arr = []
# videodata = skvideo.io.vreader(video_string)  

# count = 0 
# n = 0
# for v in videodata:
#   count += 1
#   if count % 3 == 0:
#     n+= 1
#     if n >= int(len(resized_vid_arr)*0.0) + num_frames:
#       # roi = cv2.resize(v, (400, 200))
#       # roi = roi.astype("float") / 255.0
#       # larger_vid_arr.append(roi)
#       larger_vid_arr.append(v)
# larger_vid_arr = np.array(larger_vid_arr)


In [ ]:
larger_vid_arr.shape

In [ ]:
plt.imshow(larger_vid_arr[0])
plt.show()

In [ ]:
fps = 10
def make_frame(t):
  t = int(t * fps)
  # print(t)
  return video_np[t]

from moviepy.video.io.VideoFileClip import VideoClip
myclip = VideoClip(make_frame, duration = 3)
myclip.write_videofile('larger_vid_arr_4.mp4', fps = 10)

Moviepy - Building video larger_vid_arr_4.mp4.
Moviepy - Writing video larger_vid_arr_4.mp4



Moviepy - Done !
Moviepy - video ready larger_vid_arr_4.mp4


In [ ]:
fps = 22
sr = (audio_array.shape[0]// resized_vid_arr.shape[0])*fps
print(sr, audio_np.shape)
write('5.8_transformer_clap_aud_out_4_test.mp3', sr, audio_np)

17600 (2, 24000)


In [ ]:
print(larger_vid_arr.shape)
print(audio_np.shape)

(238, 1080, 1920, 3)
(2, 24000)


In [ ]:
from moviepy.editor import *
  
clip = VideoFileClip("larger_vid_arr_4.mp4")
  
clip = clip.subclip(0, 3)
  
# loading audio file
audioclip = AudioFileClip("5.8_transformer_clap_aud_out_4_test.mp3")
  
# adding audio to the video clip
videoclip = clip.set_audio(audioclip)

videoclip.write_videofile('larger_vid_arr_with_sound_4.mp4', fps = 10)
  
# showing video clip
videoclip.ipython_display()

Moviepy - Building video larger_vid_arr_with_sound_4.mp4.
MoviePy - Writing audio in larger_vid_arr_with_sound_4TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video larger_vid_arr_with_sound_4.mp4



Moviepy - Done !
Moviepy - video ready larger_vid_arr_with_sound_4.mp4
Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
